In [7]:
# Import all necessary libraries and modules - excluding libraries for uploading files to drive
# libraries for uploading files to drive can be found in section 10
import time
from datetime import datetime
from datetime import date
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import json
import pandas as pd
import numpy as np
import re

In [37]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(chrome_options=options)
driver.get('https://www.electronicscrazy.sg/refurbished-items/laptops/')
wait = WebDriverWait(driver, 30)

C:\Users\diana.suwanto\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [38]:
# get all links in the first page
links = driver.find_elements_by_class_name('product-title')
all_link_list = []

for link in links:
    all_link_list.append(link.get_attribute('href'))

# go to the second page
next_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="pagination_contents"]/div[4]/div/a[2]/i')))
next_button.click()

time.sleep(2)
# get all links in the second page
links_2 = driver.find_elements_by_class_name('product-title')

for link in links_2:
    all_link_list.append(link.get_attribute('href'))

In [19]:
# all_link_list

In [39]:
len(all_link_list)

83

In [48]:
start = time.time()
items_dict = {}

for i in range(len(all_link_list)):
    driver.get(all_link_list[i])

    title = driver.find_element_by_xpath('//*[@id="tygh_main_container"]/div[3]/div[1]/div[2]/div[1]/div[1]/div[1]/div[2]/form/h1/bdi').text.lower()
    price = driver.find_elements_by_class_name('ty-price-num')
    # the first element is "S$"", the second element is the real dollar value
    price = price[1].text

    items_dict.update({i: {'title': title, 
                           'price': price}})

    features_label = driver.find_elements_by_class_name('ty-product-feature__label')
    features_value = driver.find_elements_by_class_name('ty-product-feature__value')

    for j in range(len(features_label)):
        label = (features_label[j].text)[:-1].lower()
        value = features_value[j].text.lower()
        items_dict[i][label] = value

    items_dict[i]['link'] = all_link_list[i]
    try:
        items_dict[i]['description'] = driver.find_element_by_xpath('//*[@id="content_description"]/div/ul').text.lower()
    except NoSuchElementException:
        print(i)
        items_dict[i]['description'] = driver.find_element_by_xpath('//*[@id="content_description"]/div/p[1]').text.lower() + \
                                        driver.find_element_by_xpath('//*[@id="content_description"]/div/p[2]').text.lower()

driver.quit()
end = time.time()
print(f'Visited {len(all_link_list)} links and recorded the data in a dictionary: items_dict. Time taken: {end-start} s.')
items_dict[0]

67
Visited 83 links and recorded the data in a dictionary: items_dict. Time taken: 231.16252541542053 s.


{'title': '(certified refurbished) acer travelmate p248-m 14-inch laptop - intel core i5-6200u cpu @2.30ghz/ 8gb ram/ 480gb ssd/ windows 10 pro',
 'price': '450.00',
 'brand': 'acer',
 'resolution': '1366 x 768',
 'screen size': '14 inches',
 'model': 'travelmate p248-m',
 'operating system': 'windows os',
 'processor': 'intel core i5-6200u',
 'processor speed': '2.3 ghz',
 'ram': '8 gb',
 'warranty': '1 month',
 'link': 'https://www.electronicscrazy.sg/certified-refurbished-acer-travelmate-p248-m-14-inch-laptop-intel-core-i5-6200u-cpu-2.30ghz-8gb-ram-480gb-ssd-windows-10-pro/',
 'description': 'certified refurbished from ec and high quality\neconomical price\nsold and shipped from singapore (hence faster delivery, easy to exchange)\n1 month warranty\n\nspecifications:\nbrand: acer\nmodel: travelmate p248-m\ndisplay: 14-inch\nprocessor/ chipset: intel core i5-6200u/ 2.30 ghz\nmemory: 8gb ram\nstorage: 480gb ssd\noperating system: windows 10 pro\nresolution: 1366 x 768 (hd)\nlcd backlig

In [49]:
df = pd.DataFrame.from_dict(items_dict, orient='index')
df.head()

,title,price,brand,resolution,screen size,model,operating system,processor,processor speed,ram,...,storage,color,product dimensions,version,package included,usb,bt version,product code,chipset,batteries
0,(certified refurbished) acer travelmate p248-m...,450.00,acer,1366 x 768,14 inches,travelmate p248-m,windows os,intel core i5-6200u,2.3 ghz,8 gb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(certified refurbished) apple macbook pro 13.3...,"1,230.00",apple,2560 x 1600,13.3 inches,NaN,mac os,intel core i5-4278u,NaN,16gb ddr3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(certified refurbished) dell latitude e5440 co...,370.00,dell,1366 x 768,14 inches,e5440,windows os,intel core i5 4th gen,1.9 ghz,4gb,...,500 gb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(certified refurbished) dell latitude e5470 14...,630.00,dell,1920 x 1080,14 inches,latitude e5470,windows os,NaN,NaN,8gb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(certified refurbished) dell latitude e6320 13...,225.00,dell,1366 x 768,13.3 inches,latitude e6320,windows os,intel core i5-2520m,2.5 ghz,4gb ddr3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# other possible website to scrape: https://laptopfactoryoutlet.com.sg/product-category/laptops/used-refurbished-secondhand-laptops/

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 0 to 82
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               83 non-null     object
 1   price               83 non-null     object
 2   brand               83 non-null     object
 3   resolution          82 non-null     object
 4   screen size         83 non-null     object
 5   model               56 non-null     object
 6   operating system    83 non-null     object
 7   processor           80 non-null     object
 8   processor speed     70 non-null     object
 9   ram                 77 non-null     object
 10  warranty            80 non-null     object
 11  link                83 non-null     object
 12  description         83 non-null     object
 13  screen type         34 non-null     object
 14  special features    17 non-null     object
 15  weight              29 non-null     object
 16  os version          8 non-nu

In [51]:
# df.to_csv("CE9010_electronicscrazy_laptop.csv")
df.to_csv("CE9010_electronicscrazy_laptop_1.csv")

## After Webscraping - Data Cleaning

In [13]:
# Read data directly
# df = pd.read_csv('CE9010_electronicscrazy_laptop.csv')
# df = pd.read_csv('CE9010_electronicscrazy_laptop_1.csv')
df = pd.read_csv('CE9010_electronicscrazy_laptop_1_drop.csv')

In [56]:
# df["resolution"].unique().tolist()

In [55]:
# df["processor"].unique().tolist()

In [58]:
df.columns

Index(['title', 'price', 'brand', 'resolution', 'screen size', 'model',
       'operating system', 'processor', 'processor speed', 'ram', 'warranty',
       'link', 'description', 'screen type', 'special features', 'weight',
       'os version', 'storage', 'color', 'product dimensions', 'version',
       'package included', 'usb', 'bt version', 'product code', 'chipset',
       'batteries'],
      dtype='object')

In [6]:
# df.drop(columns=['screen type', 'special features',
#        'os version', 'color', 'product dimensions', 'version',
#        'package included', 'usb', 'bt version', 'product code', 'chipset',
#        'batteries'], inplace = True)
df.to_csv('CE9010_electronicscrazy_laptop_1_drop.csv')

In [5]:
df['screen size'] = df['screen size'].str.replace(' inches', '')
df['warranty'] = df['warranty'].str.replace(' month', '')

In [90]:
# df['weight'].str.split('([^A-Za-z]+)([A-Za-z]+)', expand = True)
weight_all = df['weight'].str.split('([^A-Za-z]+)([A-Za-z]+)', expand = True)
weight_all[1].fillna(weight_all[0], inplace = True)
del weight_all[0]
del weight_all[3]

In [91]:
weight_all.to_csv('tryout.csv')

In [114]:
weight_all.columns = ['weight', 'unit']
weight_all['weight'].fillna(9999, inplace = True)
weight_all['weight'].astype('float')
weight_all

,weight,unit
0,9999,NaN
1,9999,NaN
2,2.2,None
3,9999,NaN
4,9999,NaN
...,...,...
78,9999,NaN
79,2.2,kg
80,9999,NaN
81,1.39,None


In [14]:
df['processor speed'] = df['processor speed'].str.replace(r'([A-Za-z]+)', '')

In [16]:
df['processor speed']

0            2.3 
1             NaN
2            1.9 
3             NaN
4            2.5 
         ...     
78           2.3 
79           2.9 
80    2.6   2.81 
81    2.6   2.81 
82           2.6 
Name: processor speed, Length: 83, dtype: object

In [17]:
df['ram'] = df['ram'].str.replace(r'([A-Za-z]+)', '')

In [18]:
df['storage'] = df['storage'].str.replace(r'([A-Za-z]+)', '')

In [21]:
df['weight'] = df['weight'].str.replace('4.9', '2.22')

In [22]:
df['weight'] = df['weight'].str.replace(r'([A-Za-z]+)', '')

In [24]:
df['resolution'].unique()

array(['1366 x 768', '2560 x 1600', '1920 x 1080', '1600 x 900',
       '1280 x 1024', '3200 x 1800', '1440 x 900', nan], dtype=object)

In [25]:
resolution_dict = {'1366 x 768':'hd',
                  '2560 x 1600': 'wqxga',
                  '1920 x 1080': 'fhd',
                  '1600 x 900': 'hd+',
                  '1280 x 1024': 'sxga',
                  '3200 x 1800': 'qhd+',
                  '1440 x 900': 'wxga+'}
df['resolution'] = df['resolution'].map(resolution_dict)

In [27]:
df.to_csv('CE9010_electronicscrazy_laptop_clean.csv')

In [29]:
df = pd.read_csv('CE9010_electronicscrazy_laptop_clean.csv')

In [30]:
df['description'][0]

'certified refurbished from ec and high quality\neconomical price\nsold and shipped from singapore (hence faster delivery, easy to exchange)\n1 month warranty\n\nspecifications:\nbrand: acer\nmodel: travelmate p248-m\ndisplay: 14-inch\nprocessor/ chipset: intel core i5-6200u/ 2.30 ghz\nmemory: 8gb ram\nstorage: 480gb ssd\noperating system: windows 10 pro\nresolution: 1366 x 768 (hd)\nlcd backlight technology: led backlight\nwidescreen display: yes\ninterfaces: usb 2.0, hdmi, vga, lan, 2 x usb 3.0, headphone/ microphone combo jack\nmemory card reader: yes (sd card)\n\naudio & video:\ngraphics processor: intel hd graphics 520\nmemory allocation technology: shared video memory (uma)\nintegrated webcam: yes\nsound: microphone'

In [38]:
df['try_ssd'] = df['description'].str.split('ssd', expand = True)[0]

In [52]:
df['try_ssd'] = df['try_ssd'].str.replace('\n', '')
df['try_ssd'] = df['try_ssd'].str.split(' ').str[-1]

In [84]:
df.to_csv('CE9010_electronicscrazy_laptop_clean_1.csv')

In [91]:
df = pd.read_csv('CE9010_electronicscrazy_laptop_clean_1.csv')

In [77]:
all_cols = df.columns.to_list()

In [78]:
unique_dict = {}
for col in all_cols:
    unique_dict[col] = df[col].unique()
#     print(col)
unique_df = pd.DataFrame.from_dict(unique_dict, orient='index')

In [79]:
# unique_df.T.to_excel('CE9010_unique.xlsx')

In [87]:
df['graphic_card'] = df['graphic_card'].str.replace('graphics', '')

In [90]:
df.to_csv('CE9010_electronicscrazy_laptop_clean_1.csv')

In [92]:
df['graphic_card'] = df['graphic_card'].str.replace('  ', ' ')

In [93]:
df.to_csv('CE9010_electronicscrazy_laptop_clean_1.csv')

# More data cleaning

In [ ]:
df1 = pd.read_csv('CE9010_electronicscrazy.csv')
df2 = pd.read_csv('CE9010_ebay.csv')
df3 = pd.read_csv('CE9010_lfol.csv')
df4 = pd.read_csv('CE9010_reebelo.csv')
df = pd.concat([df1,df2,df3,df4])
# df.to_csv('CE9010_all_data.csv')
# df.to_excel('CE9010_all_data_excel.xlsx')

In [ ]:
df.info()

In [ ]:
# buang yg level 1 model nya kosong
# cari proceesor, processor speed manually
# warranty: binary. 1: yes, 0: no
# storage = hdd + ssd
# hdd_exist = 1: yes, 0: no
# ssd_exist = 1: yes, 0: no
# assume graphic card not updated/taken off. 
# graphic_card_exist = 1: yes, 0: no
# drop operating system, level_2_model, display, emmc_gb, flash_storage_gb, weight
# resolution, screensize masukin satu2

# df[df['hdd_gb'].notnull() & df['ssd_gb'].notnull()]
# summary, yg perlu di google
# processor: 17, 
# processor speed: 95, 
# ram: 31, 
# resolution: 60, 
# screen size: 25
309-284

In [ ]:
df_null = df[df['processor'].isnull() | df['processor_speed'].isnull() | df['ram'].isnull() | df['resolution'].isnull() | df['screen_size'].isnull()]
df_null.to_excel('CE9010_DFNULL.xlsx')
df_not_null = df[df['processor'].notnull() & df['processor_speed'].notnull() & df['ram'].notnull() & df['resolution'].notnull() & df['screen_size'].notnull()]
df_not_null.to_csv('CE9010_DFNOTNULL.csv')
df_null.to_csv('CE9010_DFNULL.csv')

In [ ]:
df_null = pd.read_csv('CE9010_DFNULL_CLEANED.csv')
df_not_null = pd.read_csv('CE9010_DFNOTNULL.csv')
df = pd.concat([df_null,df_not_null])
df.columns
df.drop(['operating_system', 'level_2_model', 'display', 'emmc_gb', 'flash_storage_gb'], axis = 1, inplace = True)
# okay warranty: binary. 1: yes, 0: no
# storage = hdd + ssd
# hdd_exist = 1: yes, 0: no
# ssd_exist = 1: yes, 0: no
# graphic_card_exist = 1: yes, 0: no
df['warranty'] = df['warranty'].apply(lambda x: 0 if np.isnan(x) else 1)
df['hdd_exist'] = df['hdd_gb'].apply(lambda x: 0 if np.isnan(x) else 1)
df['ssd_exist'] = df['ssd_gb'].apply(lambda x: 0 if np.isnan(x) else 1)
df['graphic_card_exist'] = df['graphic_card'].apply(lambda x: 0 if pd.isnull(x) else 1)
df['storage'] = df['hdd_gb'].apply(lambda x: 0 if np.isnan(x) else x) + df['ssd_gb'].apply(lambda x: 0 if np.isnan(x) else x)
# df.head()
df.to_csv('CE9010_cleaned_data.csv')